## Evaluation

### Trajectory Evaluator

In [5]:
from langchain.evaluation import load_evaluator
from dotenv import load_dotenv

load_dotenv()

evaluator = load_evaluator("trajectory")

reference_steps = [
    "Fibonacci zaczyna się od 0, 1",
    "Kolejne liczby to suma dwóch poprzednich",
    "5. liczba to 5"
]

prediction_steps = [
    "Fibonacci zaczyna się od 0, 1",
    "Kolejne liczby to suma poprzednich",
    "5. liczba to 4"
]

result = evaluator.invoke({
    "prediction": prediction_steps,
    "reference": reference_steps
})


print(result)  # {'score': 0.7, 'reasoning': 'Model dobrze zaczął, ale pomylił wynik końcowy'}


ValueError: Missing some input keys: {'answer', 'agent_trajectory', 'question'}

In [ ]:
from langchain.evaluation import load_evaluator

# Ładujemy trajectory evaluator
evaluator = load_evaluator("trajectory")

# Referencja – krok po kroku
reference_steps = [
    "Fibonacci zaczyna się od 0 i 1",
    "Kolejna liczba to 1 (0+1)",
    "Następna to 2 (1+1)",
    "Kolejna to 3 (1+2)",
    "Następna to 5 (2+3)"
]

# Predykcja modelu – błędny krok w środku
prediction_steps = [
    "Fibonacci zaczyna się od 0 i 1",
    "Kolejna liczba to 1 (0+1)",
    "Następna to 2 (1+1)",
    "Kolejna to 3 (1+2)",
    "Następna to 4 (2+2)"
]

# Ewaluacja
result = evaluator.evaluate_strings(
    prediction=prediction_steps,
    reference=reference_steps
)

print(result)
